### Importing General Purpose Libraries and Datasets

###### Note: Hyperparameters for corresponsing algorithms are tuned optimally and coded accordingly

In [1]:
# Pickle: Storing Objects (Serialization)
import pickle
import gzip

# Loading USPS dataset
from PIL import Image
import os
import numpy as np

# For implementing Multilayer Perceptron Network
import keras, time
from keras.datasets import mnist
from keras.layers import Dense
from keras.models import Sequential

# For implementing confusion matrix
from sklearn.metrics import confusion_matrix

E:\ProgramFiles\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load MNIST on Python 3.x

In [2]:
filename = 'mnist.pkl.gz'
f = gzip.open(filename, 'rb')

training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
f.close()

In [3]:
# print(training_data[0].shape, test_data[0].shape , validation_data[0].shape)

### Load USPS on Python 3.x

In [4]:
USPSMat  = []
USPSTar  = []
curPath  = 'USPSdata/Numerals'
savedImg = []

for j in range(0,10):
    
# - Images in given path are stored in folders with names "0" - "9"
    curFolderPath = curPath + '/' + str(j)
    imgs =  os.listdir(curFolderPath)
    
    for img in imgs:
        curImg = curFolderPath + '/' + img
        
# ----- If given image has a png extension, resize it to 28 X 28
        if curImg[-3:] == 'png':
            img = Image.open(curImg,'r')
            img = img.resize((28, 28))
            savedImg = img
            
# --------- Normalize and invert colors
            imgdata = (255-np.array(img.getdata()))/255

# --------- Create Input and Output Vectors
            USPSMat.append(imgdata)
            USPSTar.append(j)

In [5]:
USPSMat = np.asarray(USPSMat)
USPSTar = np.asarray(USPSTar)

# One-hot encoding 
USPSMat = USPSMat.reshape(USPSMat.shape[0], -1)
USPSTar_NN = keras.utils.to_categorical(USPSTar, 10)

In [6]:
# print(USPSTar_NN.shape)

# DEEP NEURAL NETWORK

### Training on MNIST dataset

In [7]:
################### Tunable Hyperparamters ###################

# units: number of nodes
# number of hidden layers, their units & activation function
# optimizer
# batch size
# epochs

##############################################################

startNNMNIST = time.clock()
(x_train, y_train), (x_test, y_test) = mnist.load_data()
num_classes=10
image_vector_size=28*28

x_train = x_train.reshape(x_train.shape[0], image_vector_size)
x_test = x_test.reshape(x_test.shape[0], image_vector_size)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

image_size = 784
model = Sequential()

# Input Layer
model.add(Dense(units=256, activation='sigmoid', 
                input_shape=(image_size,)))

# Hidden Layer 1
model.add(Dense(units=208, activation='tanh'))

#Hidden Layer 2
model.add(Dense(units=208, activation='relu'))

#Hidden Layer 3
model.add(Dense(units=208, activation='relu'))

# Output Layer
model.add(Dense(units=num_classes, activation='softmax'))

# Add optimizers, loss functions and metrics
model.compile(optimizer='adadelta', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training data using MNIST dataset
history = model.fit(x_train, y_train, 
                    batch_size=64, 
                    epochs=50, 
                    verbose=False,
                    validation_split=.1)

time_train_NN = time.clock() - startNNMNIST

### Testing on MNIST dataset

In [8]:
startNNMNIST = time.clock()
predNNMNIST = model.predict(x_test)
time_testMNIST = time.clock() - startNNMNIST

# Confusion Matrix for Neural Network on MNIST dataset
matrixNNMNIST = confusion_matrix(y_test.argmax(axis=1), 
                                 predNNMNIST.argmax(axis=1))

# Evaluating metrics
loss1,accuracy1 = model.evaluate(x_test, y_test, 
                                 verbose=False)

### Testing on USPS dataset

In [9]:
# Testing on USPS data
startNNUSPS = time.clock()
predNNUSPS = model.predict(USPSMat)
time_testUSPS = time.clock() - startNNUSPS

# Confusion Matrix for Neural Network on USPS dataset
matrixNNUSPS = confusion_matrix(USPSTar_NN.argmax(axis=1), 
                                predNNUSPS.argmax(axis=1))

# Evaluating metrics
loss2, accuracy2 = model.evaluate(USPSMat, USPSTar_NN, 
                                  verbose=False)

### Printing Results

In [10]:
print("Training Time: %.5f s" % time_train_NN)
print("Testing Time (MNIST): %.5f s" % time_testMNIST)
print("Confusion Matrix (MNIST):")
print(matrixNNMNIST)
print("Accuracy: " + str(accuracy1*100) + " %\n\n")

print("Testing Time (USPS): %.5f s" % time_testUSPS)
print("Confusion Matrix (USPS):")
print(matrixNNUSPS)
print("Accuracy: " + str(accuracy2*100) + " %")

Training Time: 188.65216 s
Testing Time (MNIST): 0.27204 s
Confusion Matrix (MNIST):
[[ 974    0    1    1    0    1    1    1    0    1]
 [   0 1124    0    1    0    0    2    0    7    1]
 [   4    0 1005    5    3    1    3    6    5    0]
 [   0    0    4  992    1    4    0    3    5    1]
 [   0    0    3    2  960    0    4    1    1   11]
 [   1    0    0   13    0  869    5    0    3    1]
 [   5    2    0    0    5    5  936    0    4    1]
 [   1    1    9    7    0    0    0  995    2   13]
 [   1    0    2    4    2    4    0    3  955    3]
 [   1    2    0    5    7    4    0    2    4  984]]
Accuracy: 97.94 %


Testing Time (USPS): 0.42708 s
Confusion Matrix (USPS):
[[ 150    0    8  168  208  524    1   13  111  817]
 [   8    4    1   23  124   79    1    1  612 1147]
 [   2   12   58  173  265  467    0    6  312  704]
 [   0    4    1  900   17  460    0    1  251  366]
 [   2    1    0   70  309  149    0   41  602  826]
 [   6    7    7   49   68 1428    1    6  

## Importing Libraries for SVM and Random Forest

In [11]:
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
from sklearn.metrics import accuracy_score

# indices = np.arange(len(mnist.data))
# print(indices) ---> [0, 1, 2 ... 69997, 69998, 69999]

X_train, y_train = training_data
X_test, y_test = test_data
X_val, y_val = validation_data

# SVM

In [12]:
################### Tunable Hyperparamters ###################

# kernel
# C: Penalty parameter of error term
# gamma: kernel coefficient

##############################################################
'''
# Case 1
classifier1 = SVC(kernel='linear')

# Case 2
classifier1 = SVC(kernel='rbf', gamma=1)

'''
# Case 3
classifier1 = SVC(kernel='rbf')

# Training on MNIST dataset
startSVMMNIST = time.clock()
classifier1.fit(X_train[:20000], y_train[:20000])
time_train_SVM = time.clock() - startSVMMNIST

In [13]:
# Testing on MNIST dataset
startSVMMNIST = time.clock()
predSVMMNIST = classifier1.predict(X_test)
time_testMNIST = time.clock() - startSVMMNIST

# Testing on MNIST validation dataset
startSVMMNIST = time.clock()
predSVMMNIST_val = classifier1.predict(X_val)
time_valMNIST = time.clock() - startSVMMNIST

# Evaluating accuracy and confusion matrix
matrixSVMMNIST = confusion_matrix(y_test, predSVMMNIST)

accuracy3 = accuracy_score(y_test, predSVMMNIST)

In [14]:
# USPS dataset 
startSVMUSPS = time.clock()
predSVMUSPS = classifier1.predict(USPSMat)
time_testUSPS = time.clock() - startSVMUSPS

# Evaluating accuracy and confusion matrix
matrixSVMUSPS = confusion_matrix(USPSTar, predSVMUSPS)

accuracy4 = accuracy_score(USPSTar, predSVMUSPS)

In [15]:
print("Training Time: %.5f s" % time_train_SVM)
print("Testing Time (MNIST): %.5f s" % time_testMNIST)
print("Confusion Matrix (MNIST):")
print(matrixSVMMNIST)
print("Accuracy: " + str(accuracy3*100) + " %\n\n")

print("Testing Time (USPS): %.5f s" % time_testUSPS)
print("Confusion Matrix (USPS):")
print(matrixSVMUSPS)
print("Accuracy: " + str(accuracy4*100) + " %")

Training Time: 110.17562 s
Testing Time (MNIST): 76.22220 s
Confusion Matrix (MNIST):
[[ 966    0    1    0    1    5    5    1    1    0]
 [   0 1118    3    2    0    1    4    1    6    0]
 [   9    2  934   10   14    2   18   14   27    2]
 [   2    2   19  932    1   21    1   11   17    4]
 [   1    4    7    0  919    0   10    3    2   36]
 [   7    6    5   36    9  795   14    4   11    5]
 [  10    3    5    1    6   14  918    0    1    0]
 [   3   17   23    4   10    1    0  939    4   27]
 [   4    8    8   18    8   28   11    8  873    8]
 [  10    9    1   12   39    7    1   13    6  911]]
Accuracy: 93.05 %


Testing Time (USPS): 155.53797 s
Confusion Matrix (USPS):
[[ 565    4  409   18  336  223   77   37   11  320]
 [  98  455  240  124  417  190   46  397   19   14]
 [ 107   23 1345   70   48  213   67   84   31   11]
 [  71    7  164 1118   15  506    6   63   27   23]
 [  18   95   62   16 1238  245   16  162   64   84]
 [ 137   20  188  100   34 1371   69   5

# RANDOM FOREST

In [16]:
########### Tunable Hyperparameters ###########

# n_estimators: number of trees
# max_depth, max_leaf_nodes, class_weight, etc

##############################################

classifier2 = RandomForestClassifier(n_estimators=1000)

# MNIST dataset
startRFMNIST = time.clock()
classifier2.fit(X_train, y_train)
time_train_RF = time.clock() - startRFMNIST

In [17]:
# Testing on MNIST dataset
startRFMNIST = time.clock()
predRFMNIST = classifier2.predict(X_test)
time_testMNIST = time.clock() - startRFMNIST

# Evaluating accuracy and confusion matrix
matrixRFMNIST = confusion_matrix(y_test, predRFMNIST)

accuracy5 = accuracy_score(y_test, predRFMNIST)

In [18]:
# USPS dataset
startRFUSPS = time.clock()
predRFUSPS = classifier2.predict(USPSMat)
time_testUSPS = time.clock() - startRFUSPS

# Evaluating accuracy and confusion matrix
matrixRFUSPS = confusion_matrix(USPSTar, predRFUSPS)

accuracy6 = accuracy_score(USPSTar, predRFUSPS)

In [19]:
print("Training Time: %.5f s" % time_train_RF)
print("Testing Time (MNIST): %.5f s" % time_testMNIST)
print("Confusion Matrix: ")
print(matrixRFMNIST)
print("Accuracy: " + str(accuracy5*100) + " %\n\n")

print("Testing Time (USPS): %.5f s" % time_testUSPS)
print("Confusion Matrix: ")
print(matrixRFUSPS)
print("Accuracy: " + str(accuracy6*100) + " %")

Training Time: 273.89514 s
Testing Time (MNIST): 4.22343 s
Confusion Matrix: 
[[ 968    0    1    0    0    3    4    1    3    0]
 [   0 1124    2    3    0    1    3    0    1    1]
 [   6    0 1000    5    3    0    4    8    6    0]
 [   0    0   12  973    0    5    0    9    8    3]
 [   1    0    2    0  956    0    4    0    2   17]
 [   3    0    0   14    3  857    6    2    5    2]
 [   5    3    0    0    2    4  942    0    2    0]
 [   1    2   18    1    0    0    0  995    1   10]
 [   5    0    5    8    4    5    4    4  929   10]
 [   5    5    2   10   10    2    1    4    6  964]]
Accuracy: 97.08 %


Testing Time (USPS): 6.05859 s
Confusion Matrix: 
[[ 651   12  278   67  438  149   56  100    0  249]
 [  46  570  114  107   41   95   21  991   14    1]
 [  91   26 1289   67   50  189   15  265    6    1]
 [  34    6   89 1327   48  293    2  178    4   19]
 [  13  208   55   25 1075  174   15  398   20   17]
 [ 138   28  129   68   20 1480   18  109    6    4]
 [ 

# ENSEMBLE

In [20]:
# For majority voting method
from scipy import stats

In [21]:
# Converting from one-hot encoding to class labels
predNNMNIST = predNNMNIST.argmax(axis=1)
predNNUSPS = predNNUSPS.argmax(axis=1)

########## Other predictions used in ensemble ##########

# predRFMNIST, predRFUSPS, predSVMMNIST, predSVMUSPS

########################################################

In [22]:
# Combining predictions from classification algorithms
predMNIST_stacked = np.vstack((predNNMNIST, predRFMNIST, predSVMMNIST))
predUSPS_stacked = np.vstack((predNNUSPS, predRFUSPS, predSVMUSPS))

# Majority Voting - Most Frequent Class is selected
predMNIST = np.ravel(stats.mode(predMNIST_stacked).mode)
predUSPS = np.ravel(stats.mode(predUSPS_stacked).mode)

(10000,) (10000,) (10000,)


### Ensemble classifier metrics

In [23]:
# Takes results from Neural Networks, SVM and Random Forest
# Calculates class mode for every instance and assigns value
# Final class = Most frequently classified class for that instance

accuracy7 = accuracy_score(y_test, predMNIST)
matrixMNIST = confusion_matrix(y_test, predMNIST)

accuracy8 = accuracy_score(USPSTar, predUSPS)
matrixUSPS = confusion_matrix(USPSTar, predUSPS)

In [24]:
print("Confusion Matrix: ")
print(matrixMNIST)
print("Accuracy: " + str(accuracy7*100) + " %\n\n")

print("Confusion Matrix: ")
print(matrixUSPS)
print("Accuracy: " + str(accuracy8*100) + " %")

Confusion Matrix: 
[[ 970    0    1    0    0    4    3    1    1    0]
 [   0 1124    2    2    0    1    3    0    3    0]
 [   6    1  997    4    4    0    4   10    6    0]
 [   0    1    8  979    0    6    0    8    6    2]
 [   1    0    2    0  956    0    6    0    2   15]
 [   3    0    1   16    2  859    5    1    4    1]
 [   8    3    1    1    4    6  933    0    2    0]
 [   2    6   16    3    0    0    0  992    1    8]
 [   5    0    4   10    4    6    3    3  933    6]
 [   9    7    1   12   11    3    1    2    1  962]]
Accuracy: 97.05 %


Confusion Matrix: 
[[ 660   12  342   81  354  184   35   27    2  303]
 [ 100  579  196  133  394  154   38  376   20   10]
 [ 125   37 1405   77   48  195   13   84    8    7]
 [  60   11  145 1336   19  360    1   50    8   10]
 [  17  193   70   35 1199  214    5  168   47   52]
 [ 139   31  148   65   32 1520   10   44    6    5]
 [ 319   43  402   35  146  407  623   11    5    9]
 [  59  364  400  294   76  306   19  42

In [25]:
'''
REFERENCES:
1. What is the best Keras model for multi-class classification?: 
https://datascience.stackexchange.com/questions/10048/what-is-the-best-keras-model-for-multi-class-classification

2. Multi-Class Classification Tutorial with the Keras Deep Learning Library:
https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

3. Confusion Matrix in Machine Learning: 
https://www.geeksforgeeks.org/confusion-matrix-machine-learning/

4. Keras Documentation: 
https://keras.io

5. Simple Confusion Matrix Guide: 
https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/

6. Get a Confusion Matrix from a Keras Multiclass Model: 
https://stackoverflow.com/questions/50920908/get-confusion-matrix-from-a-keras-multiclass-model

7. Sci-kit Documentation: 
https://scikit-learn.org

8. How much time does SVM classifier take to train?:
https://stackoverflow.com/questions/18165213/how-much-time-does-take-train-svm-classifier
'''

'\nREFERENCES:\n1. What is the best Keras model for multi-class classification?: \nhttps://datascience.stackexchange.com/questions/10048/what-is-the-best-keras-model-for-multi-class-classification\n\n2. Multi-Class Classification Tutorial with the Keras Deep Learning Library:\nhttps://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/\n\n3. Confusion Matrix in Machine Learning: \nhttps://www.geeksforgeeks.org/confusion-matrix-machine-learning/\n\n4. Keras Documentation: \nhttps://keras.io\n\n5. Simple Confusion Matrix Guide: \nhttps://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/\n\n6. Get a Confusion Matrix from a Keras Multiclass Model: \nhttps://stackoverflow.com/questions/50920908/get-confusion-matrix-from-a-keras-multiclass-model\n\n7. Sci-kit Documentation: \nhttps://scikit-learn.org\n\n8. How much time does SVM classifier take to train?:\nhttps://stackoverflow.com/questions/18165213/how-much-time-does-take-train-svm-clas